<h1 align = 'center'>Guessing Games</h1>
<h3 align = 'center'>machine learning, one step at a time</h3>
<h3 align = 'center'>Step 10. Q-learning</h3>

**10. Q-learning**

_In Q-learning, a form of reinforcement learning, an agent develops an optimal policy based on interactions with an environment; the environment provides a series of state-action-reward sequences without any additional descriptions, labels, context, or rules._

Our last attempt at solving the maze is almost a Q-learning algorithm. It developed the policy "don't repeat your mistakes" to find a path through the maze, but it was not nearly the _optimal_ policy. An _optimal_ policy would find the exit in the fewest possible steps. When we consider what we can learn from our states, actions, and rewards, we are missing something.

Our last exercise was good at exploiting __penalties__, but what about exploiting __rewards__?

***Find the reward in this q-table:***
<pre>
=====  ================================
state         N       S       E       W

(0,0)      -839       0       0    -866
(0,1)      -241    -283       0       0
(0,2)       -67       0     -53       0
(0,3)         0       0       0       0

(1,0)         0       0    -224    -227
(1,1)         0       0       0       0
(1,2)         0     -16       0     -10
(1,3)        -3       0      -6       0

(2,0)         0     -73       0     -53
(2,1)       -12     -12     -12       0
(2,2)         0       0       0       0
(2,3)         0      __+1__      -1      -1  < -- there is the +1, over in the second column!

(3,0)         0       0       0       0
(3,1)         0       0       0       0
(3,2)         0       0       0       0
(3,3)         0       0       0       0

 </pre>
in this table, it took 3,000 random attempts to find the exit once

***There is something critically important hiding in that table.***

The table cleary says that, staring in state (2,3), a move to the South results in the state (3,3) and a reward of +1. Let's make our own notation for that:
<pre>
(2,3)[S] >>> (3,3){+1}  (the >>> means "transitions to")
</pre>
That suggests that __state (2,3) is a pretty good place to be__, because from (2,3) we can acheive a positive reward. How could we use that knowledge to favor actions that put us in (2,3)?

Let's look again at the maze, marked up to show __(2,3)[S] >>> (3,3){+1}__:
<pre>
         ...  ...  ...  +++ 
enter->  (1)  ...  ...  +++ 
         ...  ...  ...  +++ 

         ...  +++  ...  ... 
         ...  +++  ...  ... 
         ...  +++  ...  ... 

         ...  ...  +++  2,3 
         ...  ...  +++  [S]  <-this is a good state
         ...  ...  +++   +1 

         +++  +++  ...  ... 
         +++  +++  ...  3,3  <-exit
         +++  +++  ...  ... 

</pre>
That view of the maze raises an interesting question: if we are exploring, how can we _exploit_ the reward that is available _if and when we arrive at state (2,3)?_

Well... how do we ever arrive at state (2,3)? In this maze, the only way is: __(1,3)[S] >>> (2,3){0}__:
<pre>
         ...  ...  ...  +++ 
enter->  (1)  ...  ...  +++ 
         ...  ...  ...  +++ 

         ...  +++  ...  1,3 
         ...  +++  ...  [S]  <-this is a totally boring, uninteresting state
         ...  +++  ...  -0- 

         ...  ...  +++  2,3 
         ...  ...  +++  [S]  <-this is a good state
         ...  ...  +++   +1 

         +++  +++  ...  ... 
         +++  +++  ...  3,3  <-exit
         +++  +++  ...  ... 
</pre>
Wait a minute... how can (1,3) be totally boring, if it can lead us to (2,3)?

Let's take a closer look at (1,3) and (2,3), by generating a q-table that has one reward:

In [2]:
import numpy as np
from maze import Maze
maze = Maze()

def sample(maze):
    action = maze.sample()                    # this returns N,S,E,W
    return maze.action_space().index(action)  # this converts to 0,1,2,3

# build a q-table that finds the exit once
q = np.zeros((4,4,4)) 
stop = False
while not stop:
    state = maze.reset()
    done = False
    while not done:
        action = sample(maze)                         
        new_state, reward, done = maze.step(action)  
        q[state[0]][state[1]][action] += reward 
        state = new_state 
        if reward > 0:
            stop = True

Maze.print_q(q)

=====  ================================
state         N       S       E       W

(0,0)      -523       0       0    -486
(0,1)      -154    -123       0       0
(0,2)       -36       0     -37       0
(0,3)         0       0       0       0
-----  --------------------------------
(1,0)         0       0    -154    -141
(1,1)         0       0       0       0
(1,2)         0      -6       0     -11
(1,3)        -4       0      -3       0
-----  --------------------------------
(2,0)         0     -36       0     -42
(2,1)        -6      -8      -2       0
(2,2)         0       0       0       0
(2,3)         0       1       0       0
-----  --------------------------------
(3,0)         0       0       0       0
(3,1)         0       0       0       0
(3,2)         0       0       0       0
(3,3)         0       0       0       0
-----  --------------------------------


Let's take a look at (2,3):

In [3]:
print(q[2][3])

[0. 1. 0. 0.]


That should have +1 as the second entry, indicating a reward moving [S] to (3,3), which is the exit.

What about (1,3)?

In [4]:
print(q[1,3])

[-4.  0. -3.  0.]


That should have 0 as the second entry, indicating no penalty or reward for moving [S] to (2,3).

Let's say we are exploring the maze __after we have solved it once__, and we find our way to (1,3), and we randomly consider the action [S]. At that point, if we were to step [S] from (1,3), we could notice that our new state (2,3) has a maximum reward of +1... before we actually take that step.

In other words, for any possible action, we could ask _what is the maximum known reward of the resulting state, if we were to proceed with that action?_

Instead of taking note that __(1,3)[S] >>> (2,3){0}__, we could try __(1,3)[S] >>> (2,3){max(2,3)}__, which says _if we move South from (1,3), we can reasonably expect to take advantage of the maximum known reward available at the resulting state (2,3)_.

It's like saying _from past experience, I know there are cookies in the kitchen, so if a move puts me in the kitchen, I can expect to get a cookie._

What is the maximum known reward at state (2,3)?

In [5]:
print('maximum known reward at (2,3) =', max(q[2,3]))

maximum known reward at (2,3) = 1.0


It's +1... because we found the exit from (2,3) on a previous attempt. When we make notes about (1,3)[S], we should take that into account. We should exploit the rewards, not just the penalties.

And once we do that, we will be able to discover that __(1,2)[E] >>> (1,3){max(1,3)}__, and so on, backwards, all the way to the starting point:
<pre>

=== SIMPLIFIED VIEW OF HOW IT COULD WORK ===

         0,0  0,1  0,2  +++ 
enter->  [E]  [E]  [S]  +++ 
          +1   +1   +1  +++       by looking forward for previous
                                  rewards, we could find a way to
         ...  +++  1,2  1,3       propogate those rewards backwards
         ...  +++  [E]  [S]       along the optimum path to the
         ...  +++   +1   +1       solution...
                                  ...or something like that.
         ...  ...  +++  2,3       
         ...  ...  +++  [S]
         ...  ...  +++   +1 

         +++  +++  ...  ... 
         +++  +++  ...  3,3  <-exit
         +++  +++  ...  ... 
         
=== IT IS ALMOST THIS SIMPLE IN PRACTICE ===
</pre>
("or something like that" means there a few details yet to discover)

<hr>
***Exercises***<p>
Explore the maze by exploiting rewards and penalites:
- Alter the program to cause a positive reward to be noted for state (0,0)[E] by taking into account the maximum known reward for each step.
- Accumulate enough positive rewards to find an optimal solution to the maze.
- Discover that this approach works sometimes, but not always, by running it several times
- Why doesn't this always work?

In [ ]:
import numpy as np
from maze import Maze

'''
=== DO NOT CHANGE CODE STARTING HERE ====== THOU SHALT NOT PASS ================================
'''
maze = Maze()
def sample(maze):
    action = maze.sample()                   
    return maze.action_space().index(action)

# follow the optimum path in the q-table, but don't take more than 12 steps
def walk(q):
    maze = Maze()
    state = maze.reset()
    done = False
    steps = 0
    while not done and steps <= 12:
        state, reward, done = maze.step(np.argmax(q[state[0]][state[1]]))
        print(steps, maze)
        steps += 1
'''
=== AND ENDING HERE ======================= THOU SHALT NOT PASS ================================
'''
q = np.zeros((4,4,4))
solved = False
while not solved:
    state = maze.reset()
    done = False
    while not done:
        action = sample(maze)                         
        new_state, reward, done = maze.step(action) 
        
        ########################################################################################
        #                                                                                      #
        #  YOUR CODE GOES HERE...                                                              #
        #                                                                                      #
        #  row = state[0], col = state[1]                                                      #
        #  do the usual update: q[row][col][action] += reward                                  #
        #  then... if the new_state is in bounds:                                              #
        #      adjust q[row][col][action] by the maximum reward available at new_state         #
        #                                                                                      #
        ########################################################################################
 
        state = new_state
        solved = True if max(q[0][0]) > 0 else False

Maze.print_q(q)
walk(q)